# **Coletando Dados**

Os dados foram disponibilizados em meu github público. Eu subi esses arquivos no github e estou usando esse script pronto para carregar esses arquivos no Google Colab.

In [ ]:
import requests
import zipfile
import io
import os

# URL do arquivo .zip no GitHub
url = "https://github.com/geeklicantropo/MESTRADO_PUBLICO/raw/main/DATA_MINING/TAREFAS/Tarefa2/store-sales-time-series-forecasting.zip"

# Pasta onde os arquivos serão extraídos
extract_to = "/content/"

# Baixar o arquivo .zip
response = requests.get(url)
zip_file_path = os.path.join(extract_to, "store-sales-time-series-forecasting.zip")

# Verificar se a resposta foi bem sucedida
if response.status_code == 200:
    # Salvar o arquivo .zip no disco
    with open(zip_file_path, "wb") as f:
        f.write(response.content)

    # Verificar se o arquivo salvo é um zip válido
    try:
        with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
            zip_ref.extractall(extract_to)
        print("Arquivos extraídos com sucesso!")
    except zipfile.BadZipFile:
        print("Erro: O arquivo baixado não é um arquivo ZIP válido.")

    # Remover o arquivo .zip baixado
    os.remove(zip_file_path)
else:
    print(f"Erro ao baixar o arquivo: {response.status_code}")


# **Objetivo: Prever os preços das próximas vendas com a menor margem de erro possível**

**Obs.:**

* O Equador é um país cuja economia é altamente dependente do petróleo, tornando-se vulnerável a choques nos preços desse recurso.

* Os salários do setor público são pagos quinzenalmente, nos dias 15 e 30, o que pode influenciar as vendas nos supermercados.

* Um terremoto de magnitude 7,8 atingiu o Equador em 16 de abril de 2016. Esforços de socorro envolveram a doação de água e produtos de primeira necessidade, impactando significativamente as vendas nos supermercados por várias semanas após o evento.

In [1]:
#Importações de bibliotecas para manipulação de dados
import pandas as pd
import numpy as np
import gc
import os
from datetime import timedelta

#Importações de bibliotecas para visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport


#Importações de bibliotecas para processamento e validação de dados
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, MinMaxScaler, StandardScaler
from sklearn.model_selection import TimeSeriesSplit, train_test_split, cross_val_score

#Importações de bibliotecas para modelagem de dados
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

#Importações de bibliotecas para métricas e otimização de modelos
from sklearn.metrics import mean_squared_error, make_scorer
import optuna

#Importações de bibliotecas para utilidades adicionais e paralelização
from tqdm import tqdm, trange
import pickle
import math
import multiprocessing
from joblib import Parallel, delayed
import traceback

#Configuração de exibição do pandas
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

d:\POS_GRADUACAO\neoenv\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


<h3>Feature Engineering</h3>

In [2]:
''' 
Função para salvar um dataFrame em CSV e liberar a memória utilizada
Essa função foi criada porque fazer o merge de todos os datasets em 
memória estava quebrando o Kernel e reiniciando o Jupyter Notebook.
'''
def save_and_cleanup(df, file_name):
    #Cria o diretório pai se não existir
    parent_dir = os.path.dirname(file_name)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    #Salva o dataFrame em CSV
    df.to_csv(file_name, index=False)
    #Libera a memória
    del df
    gc.collect()
    print(f"Saved and cleaned up: {file_name}")

#Função para carregar um CSV e mergear com um dataFrame base
def load_and_merge(base_df, file_name, on_columns, how='left'):
    # Carrega o CSV
    df = pd.read_csv(file_name)
    #Converte a coluna 'date' para datetime
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
    #Mergeia o dataFrame base com o CSV carregado
    merged_df = base_df.merge(df, on=on_columns, how=how)
    #Libera a memória
    del base_df, df
    gc.collect()
    print(f"Merged with: {file_name}")
    return merged_df

#Função para realizar a engenharia de features
def feature_engineering(data, transactions_path, stores_path, holidays_events_path, oil_path, temp_dir='temp'):
    #Data do terremoto
    earthquake_date = pd.to_datetime('2016-04-16')
    #Dias de pagamento de salários
    wage_payment_days = [15, 30, 31]

    print("Loading and preprocessing initial data...")
    data = data.copy()
    data['date'] = pd.to_datetime(data['date'])
    data['onpromotion'] = data['onpromotion'].astype('float32')
    data['store_nbr'] = data['store_nbr'].astype('int8')

    #Remove a coluna 'id' se existir
    if 'id' in data.columns:
        data = data.drop(columns=['id'])

    save_and_cleanup(data, f"{temp_dir}/data.csv")

    print("Loading and saving transactions...")
    #Carrega o CSV de transações
    transactions = pd.read_csv(transactions_path)
    transactions['date'] = pd.to_datetime(transactions['date'])
    transactions['store_nbr'] = transactions['store_nbr'].astype('int8')
    #Cria a coluna 'month_day' com o formato mês-dia
    transactions['month_day'] = transactions['date'].dt.strftime('%m-%d')
    #Agrupa as transações por 'store_nbr' e 'month_day', calculando a média
    transactions_grouped = transactions.groupby(['store_nbr', 'month_day']).agg({'transactions': 'mean'}).reset_index()
    save_and_cleanup(transactions_grouped, f"{temp_dir}/transactions_grouped.csv")

    print("Loading and saving stores...")
    #Carrega o CSV de lojas
    stores = pd.read_csv(stores_path, dtype={'store_nbr': 'int8', 'cluster': 'int8'})
    save_and_cleanup(stores, f"{temp_dir}/stores.csv")

    print("Loading and saving holidays_events...")
    #Carrega o CSV de feriados
    holidays_events = pd.read_csv(holidays_events_path)
    holidays_events['date'] = pd.to_datetime(holidays_events['date'])
    save_and_cleanup(holidays_events, f"{temp_dir}/holidays_events.csv")

    print("Loading, processing, and saving oil data...")
    #Carrega o CSV de preços do petróleo ao longo do tempo
    oil = pd.read_csv(oil_path)
    oil['date'] = pd.to_datetime(oil['date'])
    #Completa os preços de petróleo faltantes em algumas datas com interpolação
    oil_dates = pd.date_range(start=oil['date'].min(), end=oil['date'].max())
    oil_df_full = pd.DataFrame({'date': oil_dates})
    oil_df_full = oil_df_full.merge(oil, on='date', how='left')
    oil_df_full['dcoilwtico'] = oil_df_full['dcoilwtico'].interpolate()
    oil_df_full['dcoilwtico'] = oil_df_full['dcoilwtico'].fillna(method='bfill').fillna(method='ffill')
    save_and_cleanup(oil_df_full, f"{temp_dir}/oil_df_full.csv")

    print("Loading data and merging transactions...")
    #Carrega o CSV de dados e converte a coluna 'date' para datetime
    merged_df = pd.read_csv(f"{temp_dir}/data.csv")
    merged_df['date'] = pd.to_datetime(merged_df['date'])
    merged_df['month_day'] = merged_df['date'].dt.strftime('%m-%d')
    #Carrega o CSV de transações agrupadas e converte a coluna 'month_day' para string
    transactions_grouped = pd.read_csv(f"{temp_dir}/transactions_grouped.csv")
    transactions_grouped['month_day'] = transactions_grouped['month_day'].astype(str)
    merged_df['month_day'] = merged_df['month_day'].astype(str)
    #Mergeia os dados com as transações agrupadas
    merged_df = load_and_merge(merged_df, f"{temp_dir}/transactions_grouped.csv", on_columns=['store_nbr', 'month_day'], how='left')
    merged_df.drop(columns=['month_day'], inplace=True)

    save_and_cleanup(merged_df, f"{temp_dir}/merged_data_transactions.csv")

    print("Loading merged data and merging stores...")
    #Carrega o CSV de dados mergeados com transações e converte a coluna 'date' para datetime
    merged_df = pd.read_csv(f"{temp_dir}/merged_data_transactions.csv")
    merged_df['date'] = pd.to_datetime(merged_df['date'])
    #Mergeia os dados com o CSV de lojas
    merged_df = load_and_merge(merged_df, f"{temp_dir}/stores.csv", on_columns=['store_nbr'], how='left')

    save_and_cleanup(merged_df, f"{temp_dir}/merged_data_stores.csv")

    print("Loading merged data and merging holidays_events...")
    #Carrega o CSV de dados mergeados com lojas e converte a coluna 'date' para datetime
    merged_df = pd.read_csv(f"{temp_dir}/merged_data_stores.csv")
    merged_df['date'] = pd.to_datetime(merged_df['date'])
    #Mergeia os dados com o CSV de feriados
    merged_df = load_and_merge(merged_df, f"{temp_dir}/holidays_events.csv", on_columns=['date'], how='left')

    save_and_cleanup(merged_df, f"{temp_dir}/merged_data_holidays_events.csv")

    print("Loading merged data and merging oil_df_full...")
    #Carrega o CSV de dados mergeados com feriados e converte a coluna 'date' para datetime
    merged_df = pd.read_csv(f"{temp_dir}/merged_data_holidays_events.csv")
    merged_df['date'] = pd.to_datetime(merged_df['date'])
    #Mergeia os dados com o CSV de preços de petróleo completo
    merged_df = load_and_merge(merged_df, f"{temp_dir}/oil_df_full.csv", on_columns=['date'], how='left')
    merged_df.rename(columns={'type_x': 'store_type', 'type_y': 'holiday_type'}, inplace=True)

    #Adiciona clusters de feriados e outras features adicionais
    print("Adding holiday clusters and additional features...")
    holiday_clusters = {
        'Foundation Days': [
            'Fundacion de Manta', 'Fundacion de Cuenca', 'Fundacion de Machala', 'Fundacion de Santo Domingo', 'Fundacion de Esmeraldas',
            'Fundacion de Riobamba', 'Fundacion de Ambato', 'Fundacion de Ibarra', 'Fundacion de Quito-1', 'Fundacion de Quito',
            'Fundacion de Loja', 'Fundacion de Guayaquil-1', 'Fundacion de Guayaquil', 'Traslado Fundacion de Guayaquil',
            'Traslado Fundacion de Quito'
        ],
        'Provincialization Days': [
            'Provincializacion de Cotopaxi', 'Provincializacion de Imbabura', 'Provincializacion de Santo Domingo', 'Provincializacion Santa Elena'
        ],
        'Cantonization Days': [
            'Cantonizacion de Libertad', 'Cantonizacion de Riobamba', 'Cantonizacion del Puyo', 'Cantonizacion de Guaranda', 'Cantonizacion de Latacunga',
            'Cantonizacion de El Carmen', 'Cantonizacion de Cayambe', 'Cantonizacion de Quevedo', 'Cantonizacion de Salinas'
        ],
        'Independence Days': [
            'Primer Grito de Independencia', 'Independencia de Guayaquil', 'Independencia de Cuenca', 'Independencia de Guaranda',
            'Independencia de Latacunga', 'Independencia de Ambato', 'Traslado Independencia de Guayaquil', 'Traslado Primer Grito de Independencia'
        ],
        'Religious Holidays': [
            'Navidad-4', 'Navidad-3', 'Navidad-2', 'Puente Navidad', 'Navidad-1', 'Navidad', 'Navidad+1', 'Puente Primer dia del ano',
            'Primer dia del ano-1', 'Primer dia del ano', 'Viernes Santo'
        ],
        'Natural Disasters': [
            'Terremoto Manabi', 'Terremoto Manabi+1', 'Terremoto Manabi+2', 'Terremoto Manabi+3', 'Terremoto Manabi+4', 'Terremoto Manabi+5',
            'Terremoto Manabi+6', 'Terremoto Manabi+7', 'Terremoto Manabi+8', 'Terremoto Manabi+9', 'Terremoto Manabi+10', 'Terremoto Manabi+11',
            'Terremoto Manabi+12', 'Terremoto Manabi+13', 'Terremoto Manabi+14', 'Terremoto Manabi+15', 'Terremoto Manabi+16', 'Terremoto Manabi+17',
            'Terremoto Manabi+18', 'Terremoto Manabi+19', 'Terremoto Manabi+20', 'Terremoto Manabi+21', 'Terremoto Manabi+22', 'Terremoto Manabi+23',
            'Terremoto Manabi+24', 'Terremoto Manabi+25', 'Terremoto Manabi+26', 'Terremoto Manabi+27', 'Terremoto Manabi+28', 'Terremoto Manabi+29',
            'Terremoto Manabi+30'
        ],
        'Public Holidays': [
            'Dia de Difuntos', 'Carnaval', 'Dia del Trabajo', 'Dia de la Madre-1', 'Dia de la Madre', 'Puente Dia de Difuntos', 'Recupero Puente Dia de Difuntos', 'Traslado Primer dia del ano'
        ],
        'Historical Battles and Commemorations': [
            'Batalla de Pichincha', 'Traslado Batalla de Pichincha'
        ],
        'Sports Events': [
            'Inauguracion Mundial de futbol Brasil', 'Mundial de futbol Brasil: Ecuador-Suiza', 'Mundial de futbol Brasil: Ecuador-Honduras',
            'Mundial de futbol Brasil: Ecuador-Francia', 'Mundial de futbol Brasil: Octavos de Final', 'Mundial de futbol Brasil: Cuartos de Final',
            'Mundial de futbol Brasil: Semifinales', 'Mundial de futbol Brasil: Tercer y cuarto lugar', 'Mundial de futbol Brasil: Final'
        ],
        'Shopping Events': [
            'Black Friday', 'Cyber Monday'
        ],
        'Recovery Days': [
            'Recupero puente Navidad', 'Recupero puente primer dia del ano', 'Recupero Puente Navidad', 'Recupero Puente Primer dia del ano'
        ]
    }

    #Função para mapear a descrição do feriado para seu cluster correspondente
    def map_holiday_cluster(description):
        for cluster, holidays in holiday_clusters.items():
            if description in holidays:
                return cluster
        raise ValueError(f"Unmapped holiday description: {description}")

    #Mapeia a coluna 'description' para o cluster de feriados
    merged_df['holiday_cluster'] = merged_df['description'].apply(lambda x: map_holiday_cluster(x) if pd.notna(x) else np.nan)

    #Adiciona a coluna 'day_of_week' ao DataFrame
    merged_df['day_of_week'] = merged_df['date'].dt.dayofweek

    #Adiciona a coluna 'is_weekend' indicando se o dia é fim de semana
    merged_df['is_weekend'] = merged_df['day_of_week'].isin([5, 6]).astype(int)  # 5 = Saturday, 6 = Sunday
    merged_df['month'] = merged_df['date'].dt.month
    merged_df['year'] = merged_df['date'].dt.year
    merged_df['week'] = merged_df['date'].dt.isocalendar().week

    #Adiciona a coluna 'is_holiday' indicando se o dia é feriado
    if 'holiday_type' in merged_df.columns:
        merged_df['is_holiday'] = merged_df['holiday_type'].notna().astype(int)
    else:
        merged_df['is_holiday'] = 0

    #Adiciona a coluna 'is_wage_payment_day' indicando se o dia é dia de pagamento de salários
    merged_df['is_wage_payment_day'] = merged_df['date'].dt.day.isin(wage_payment_days).astype(int)
    #Adiciona a coluna 'earthquake_impact' indicando o impacto do terremoto
    merged_df['earthquake_impact'] = ((merged_df['date'] >= earthquake_date) & (merged_df['date'] <= earthquake_date + timedelta(days=30))).astype(int)

    #Mapeia a coluna 'family' para seu cluster correspondente
    cluster_mapping = {
        'AUTOMOTIVE': 'Automotive and Hardware',
        'BABY CARE': 'Personal and Home Care',
        'BEAUTY': 'Personal and Home Care',
        'BEVERAGES': 'Beverages',
        'BOOKS': 'Entertainment and Leisure',
        'BREAD/BAKERY': 'Food',
        'CELEBRATION': 'Celebration and Seasonal',
        'CLEANING': 'Personal and Home Care',
        'DAIRY': 'Food',
        'DELI': 'Food',
        'EGGS': 'Food',
        'FROZEN FOODS': 'Food',
        'GROCERY I': 'Food',
        'GROCERY II': 'Food',
        'HARDWARE': 'Automotive and Hardware',
        'HOME AND KITCHEN I': 'Home and Kitchen',
        'HOME AND KITCHEN II': 'Home and Kitchen',
        'HOME APPLIANCES': 'Home and Kitchen',
        'HOME CARE': 'Personal and Home Care',
        'LADIESWEAR': 'Clothing and Accessories',
        'LAWN AND GARDEN': 'Celebration and Seasonal',
        'LINGERIE': 'Clothing and Accessories',
        'LIQUOR,WINE,BEER': 'Beverages',
        'MAGAZINES': 'Entertainment and Leisure',
        'MEATS': 'Food',
        'PERSONAL CARE': 'Personal and Home Care',
        'PET SUPPLIES': 'Pet Supplies',
        'PLAYERS AND ELECTRONICS': 'Entertainment and Leisure',
        'POULTRY': 'Food',
        'PREPARED FOODS': 'Food',
        'PRODUCE': 'Food',
        'SCHOOL AND OFFICE SUPPLIES': 'School and Office Supplies',
        'SEAFOOD': 'Food'
    }
    #Mapeia a coluna 'family' para 'family_cluster'
    merged_df['family_cluster'] = merged_df['family'].map(cluster_mapping)
    #Remove as colunas 'family' e 'description', já que já temos seus clusters
    merged_df = merged_df.drop(columns=['family', 'description'])

    #Move a coluna 'sales' para o final do DataFrame
    if 'sales' in merged_df.columns:
        cols = [col for col in merged_df.columns if col != 'sales'] + ['sales']
        merged_df = merged_df[cols]

    #O dataframe final é salvo em CSV
    save_and_cleanup(merged_df, f"{temp_dir}/merged_df_final.csv")

    return merged_df

<h3>EDA</h3>
<p>Realiza análises estatísticas automáticas usando a biblioteca ProfileReport. É uma ferramenta que provê inúmeros insights sobre os dados de maneira rápida e automática.</p>

In [ ]:
#Classe para realizar Análise Exploratória de Dados (EDA)
class EDA:
    def __init__(self, df):
        self.df = df.copy()
        self._preprocess()

    #Função para pré-processar os dados
    def _preprocess(self):
        #Preenche valores nulos nas colunas de feriados
        holiday_columns = ['holiday_type', 'locale', 'locale_name', 'transferred', 'holiday_cluster']
        for col in holiday_columns:
            if col in self.df.columns:
                if self.df[col].dtype == 'object':
                    self.df[col] = self.df[col].fillna('No_Holiday')
                elif self.df[col].dtype == 'bool':
                    self.df[col] = self.df[col].fillna(False)
                else:
                    self.df[col] = self.df[col].fillna(0)

        #Preenche valores nulos na coluna transactions com zero
        if 'transactions' in self.df.columns:
            self.df['transactions'] = self.df['transactions'].fillna(0)

        #Substitui os valores numéricos de day_of_week por nomes dos dias
        day_of_week_mapping = {0: 'domingo', 1: 'segunda-feira', 2: 'terça-feira', 3: 'quarta-feira',
                               4: 'quinta-feira', 5: 'sexta-feira', 6: 'sábado'}
        if 'day_of_week' in self.df.columns:
            self.df['day_of_week'] = self.df['day_of_week'].map(day_of_week_mapping)

        #Substitui os valores numéricos de month por nomes dos meses
        month_mapping = {1: 'janeiro', 2: 'fevereiro', 3: 'março', 4: 'abril', 5: 'maio', 6: 'junho',
                         7: 'julho', 8: 'agosto', 9: 'setembro', 10: 'outubro', 11: 'novembro', 12: 'dezembro'}
        if 'month' in self.df.columns:
            self.df['month'] = self.df['month'].map(month_mapping)

    #Função para gerar o relatório de EDA
    def generate_report(self):
        profile = ProfileReport(self.df, title="EDA Report", explorative=True)
        profile.to_notebook_iframe()
        return profile


In [ ]:

test_path = 'test.csv'
train_path = 'train.csv'
transactions_path = 'transactions.csv'
stores_path = 'stores.csv'
holidays_events_path = 'holidays_events.csv'
oil_path = 'oil.csv'
parquet_path = 'processed_data.parquet'

train = pd.read_csv(train_path)

df_EDA = feature_engineering(train, transactions_path, stores_path, holidays_events_path, oil_path, temp_dir='train_temp')

In [ ]:

#Cria a instância da classe EDA e gera o relatório
eda = EDA(df_EDA)
report = eda.generate_report()

# **Testando alguns modelos**

<h3>Preparar os dados para rodar os modelos</h3>
<p>Mesmo depois dos dados passarem por feature engineering, eles ainda precisam de algum tratamento para poderem ser utilizados nos modelos. Isso acontece porque muitos modelos não aceitam valores nulos, categóricos ou mesmo valores contínuos cujo range para a mesma feature é muito grande. Como o tratamento de treino e de teste precisam ser feitos da mesma forma, do contrário, causará um erro na hora do treinamento, precisamos padronizar esse tratamento através das funções abaixo </p>

In [3]:
from sklearn.preprocessing import LabelEncoder


In [4]:
# Prepara os dados de treino para a modelagem
def prepare_data_train(df, target_var, test_size=0.2):
    df['day_of_week'] = df['date'].dt.dayofweek  # Adiciona a coluna day_of_week a partir da coluna date

    categorical_features = df.select_dtypes(include=['object', 'category', 'bool']).columns
    numeric_features = df.select_dtypes(include=[np.number]).columns.drop(target_var)

    df[categorical_features] = df[categorical_features].astype(str)

    holiday_columns = ['holiday_type', 'locale', 'locale_name', 'transferred', 'holiday_cluster']
    for col in holiday_columns:
        if col in df.columns:
            if df[col].dtype == 'object':
                df[col] = df[col].fillna('No_Holiday')
            elif df[col].dtype == 'bool':
                df[col] = df[col].fillna(False)
            else:
                df[col] = df[col].fillna(0)

    imputer_numeric = SimpleImputer(strategy='mean')
    df[numeric_features] = imputer_numeric.fit_transform(df[numeric_features])

    # Codificação de variáveis categóricas usando LabelEncoder
    encoders = {}
    for col in categorical_features:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        encoders[col] = le

    X = df.drop(columns=[target_var])
    y = df[target_var]

    train_size = int(len(X) * (1 - test_size))
    X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
    y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

    assert len(X_train) == len(y_train), "Tamanho inconsistente entre X_train e y_train"
    assert len(X_test) == len(y_test), "Tamanho inconsistente entre X_test e y_test"

    train_columns = X.columns  # Captura as colunas do conjunto de treino antes da transformação

    scaler = RobustScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=train_columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=train_columns)

    y_scaler = MinMaxScaler()
    y_train = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    y_test = y_scaler.transform(y_test.values.reshape(-1, 1)).flatten()

    return X_train, X_test, y_train, y_test, encoders, imputer_numeric, scaler, y_scaler, train_columns

# Prepara os dados de teste para a modelagem
def prepare_data_test(df, encoders, imputer_numeric, scaler, train_columns):
    df['day_of_week'] = df['date'].dt.dayofweek  # Adiciona a coluna day_of_week a partir da coluna date
    categorical_features = df.select_dtypes(include=['object', 'category', 'bool']).columns
    numeric_features = df.select_dtypes(include=[np.number]).columns

    df[categorical_features] = df[categorical_features].astype(str)

    holiday_columns = ['holiday_type', 'locale', 'locale_name', 'transferred', 'holiday_cluster']
    for col in holiday_columns:
        if col in df.columns:
            if df[col].dtype == 'object':
                df[col] = df[col].fillna('No_Holiday')
            elif df[col].dtype == 'bool':
                df[col] = df[col].fillna(False)
            else:
                df[col] = df[col].fillna(0)

    df[numeric_features] = imputer_numeric.transform(df[numeric_features])

    for col, encoder in encoders.items():
        df[col] = encoder.transform(df[col])

    df_encoded = pd.DataFrame(scaler.transform(df), columns=train_columns)

    return df_encoded



<h3>Rodando e comparando diferentes modelos usando Optuna</h3>
<p>Logo abaixo, 4 modelos serão otimizados (Grid Search com Optuna) e treinados usando uma validação cruzada. Os modelos a serem treinados são: Linear Regression, Random Forest, XGBoost e LightGBM. Depois terão suas métricas 'Root Mean Squared Logarithmic Error' comparadas visualmente </p>
<p>Devido a enorme quantidade de tempo que leva para fazer a validação cruzada dos 4 modelos, eu optei pela estratégia de testar as otimizações procurando pela melhor performance e salvar os resultados em disco. Daí se acontecesse algum problema durante a execução, eu não iria precisar retreinar os modelos que já foram treinados, bastando recuperá-los e carregá-los em memória.</p>

In [5]:
#Função para calcular o Root Mean Squared Logarithmic Error (RMSLE)def rmsle(y_true, y_pred):
def rmsle(y_true, y_pred):
    y_true = np.where(y_true <= 0, 0, y_true)
    y_pred = np.where(y_pred <= 0, 0, y_pred)
    
    if not np.all(np.isfinite(y_true)) or not np.all(np.isfinite(y_pred)):
        raise ValueError("Valores infinitos ou muito grandes encontrados em y_true ou y_pred")
    
    log_y_true = np.log1p(y_true)
    log_y_pred = np.log1p(y_pred)
    
    mse = mean_squared_error(log_y_true, log_y_pred)
    return np.sqrt(mse)

# Função para otimizar os hiperparâmetros usando Optuna
def optimize_model(trial, model_name, X_train, y_train):
    print(f"Otimização dos hiperparâmetros para {model_name}...")
    
    if model_name == 'Random Forest':
        n_estimators = trial.suggest_int('n_estimators', 10, 300)
        max_depth = trial.suggest_int('max_depth', 3, 50)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42,
            n_jobs=-1
        )
    elif model_name == 'XGBoost':
        n_estimators = trial.suggest_int('n_estimators', 10, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
        max_depth = trial.suggest_int('max_depth', 3, 50)
        subsample = trial.suggest_float('subsample', 0.5, 1.0)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
        model = XGBRegressor(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            random_state=42,
            n_jobs=-1
        )
    elif model_name == 'LightGBM':
        n_estimators = trial.suggest_int('n_estimators', 10, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
        max_bin = trial.suggest_int('max_bin', 100, 300)
        bagging_fraction = trial.suggest_float('bagging_fraction', 0.5, 1.0)
        feature_fraction = trial.suggest_float('feature_fraction', 0.5, 1.0)
        num_leaves = trial.suggest_int('num_leaves', 20, 150)
        tweedie_variance_power = trial.suggest_float('tweedie_variance_power', 1.0, 2.0)
        model = LGBMRegressor(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_bin=max_bin,
            bagging_fraction=bagging_fraction,
            feature_fraction=feature_fraction,
            num_leaves=num_leaves,
            tweedie_variance_power=tweedie_variance_power,
            objective='tweedie',
            random_state=42,
            force_row_wise=True,
            n_jobs=-1
        )

    tscv = TimeSeriesSplit(n_splits=5)
    scores = []

    if isinstance(X_train, pd.DataFrame):
        X_train = X_train.to_numpy()
    if isinstance(y_train, pd.Series):
        y_train = y_train.to_numpy()

    for train_idx, val_idx in tqdm(tscv.split(X_train), desc=f"Cross-validation para {model_name}"):
        X_t, X_v = X_train[train_idx], X_train[val_idx]
        y_t, y_v = y_train[train_idx], y_train[val_idx]

        model.fit(X_t, y_t)
        y_pred = model.predict(X_v)
        score = np.sqrt(mean_squared_error(y_v, y_pred))
        scores.append(score)

    mean_score = np.mean(scores)
    return mean_score

# Função para plotar a comparação de desempenho dos modelos
def plot_model_comparisons(model_results):
    model_names = list(model_results.keys())
    rmsle_values = list(model_results.values())

    plt.figure(figsize=(12, 6))
    sns.barplot(x=model_names, y=rmsle_values, palette="viridis")
    plt.xlabel('Modelos')
    plt.ylabel('RMSLE')
    plt.title('Comparação de Desempenho dos Modelos')
    plt.xticks(rotation=45)
    plt.show()

# Função para plotar as previsões do modelo
def plot_predictions(y_test, y_pred, model_name, y_scaler, dates):
    y_test = y_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()

    df_y_test = pd.DataFrame({'date': dates, 'y_test': y_test})
    df_y_pred = pd.DataFrame({'date': dates, 'y_pred': y_pred})

    df_y_test_daily = df_y_test.groupby('date').mean().reset_index()
    df_y_pred_daily = df_y_pred.groupby('date').mean().reset_index()

    plt.figure(figsize=(12, 6))
    plt.plot(df_y_test_daily['date'], df_y_test_daily['y_test'], label='Real')
    plt.plot(df_y_pred_daily['date'], df_y_pred_daily['y_pred'], label='Previsto')
    plt.title(f'Previsões com {model_name}')
    plt.xlabel('Data')
    plt.ylabel('Vendas Médias Diárias')
    plt.legend()
    plt.show()


#Função para executar os modelos otimizados e comparar suas performances
def run_optimized_models(df, target_var='sales', save_dir='model_results'):
    # Cria o diretório para salvar os resultados dos modelos se não existir
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    print("Preparando os dados...")
    #Prepara os dados de treino e teste
    X_train, X_test, y_train, y_test, encoders, imputer_numeric, scaler, y_scaler, train_columns = prepare_data_train(df, target_var)
    #Extrai as datas para o conjunto de teste
    dates = df.iloc[X_test.index]['date']  

    print("Verificando colunas após preparação dos dados...")
    print(f"Colunas do conjunto de treino: {list(X_train.columns)}")
    print(f"Colunas do conjunto de teste: {list(X_test.columns)}")

    print("Iniciando otimização dos hiperparâmetros...")
    #Definições dos modelos e suas funções de otimização
    model_definitions = {
        'Random Forest': ('study_rf.pkl', lambda trial: optimize_model(trial, 'Random Forest', X_train, y_train)),
        'XGBoost': ('study_xgb.pkl', lambda trial: optimize_model(trial, 'XGBoost', X_train, y_train)),
        'LightGBM': ('study_lgbm.pkl', lambda trial: optimize_model(trial, 'LightGBM', X_train, y_train))
    }

    best_params = {}
    for model_name, (file_name, opt_function) in model_definitions.items():
        model_path = os.path.join(save_dir, file_name)
        if os.path.exists(model_path):
            print(f"{model_name} já foi otimizado. Pulando para o próximo.")
            with open(model_path, 'rb') as f:
                study = pickle.load(f)
            best_params[model_name] = study.best_params
        else:
            study = optuna.create_study(direction='minimize')
            try:
                study.optimize(opt_function, n_trials=10)
                best_params[model_name] = study.best_params
                with open(model_path, 'wb') as f:
                    pickle.dump(study, f)
            except Exception as e:
                print(f"Erro ao otimizar {model_name}: {e}")
                with open(os.path.join(save_dir, f'{model_name}_optimization_error.log'), 'w') as f:
                    f.write(traceback.format_exc())
                raise e

    #Inicializa os modelos com os melhores hiperparâmetros encontrados
    models = {
        'Regressão Linear': LinearRegression(),
        'Random Forest': RandomForestRegressor(**best_params['Random Forest'], random_state=42, n_jobs=-1),
        'XGBoost': XGBRegressor(**best_params['XGBoost'], random_state=42, n_jobs=-1),
        'LightGBM': LGBMRegressor(**best_params['LightGBM'], random_state=42, n_jobs=-1, force_row_wise=True)
    }

    model_results = {}

    #Treina e avalia cada modelo
    for model_name, model in tqdm(models.items(), desc="Treinando modelos"):
        try:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            y_pred = np.maximum(y_pred, 0)
            rmsle_value = rmsle(y_test, y_pred)
            print(f'{model_name} RMSLE: {rmsle_value}')
            model_results[model_name] = rmsle_value
            plot_predictions(y_test, y_pred, model_name, y_scaler, dates)

            #Salva o modelo treinado e as previsões
            with open(os.path.join(save_dir, f'{model_name}_model.pkl'), 'wb') as f:
                pickle.dump(model, f)
            np.save(os.path.join(save_dir, f'{model_name}_y_pred.npy'), y_pred)
            np.save(os.path.join(save_dir, f'{model_name}_y_test.npy'), y_test)
        except Exception as e:
            print(f"Erro ao treinar {model_name}: {e}")
            with open(os.path.join(save_dir, f'{model_name}_training_error.log'), 'w') as f:
                f.write(traceback.format_exc())
            raise e

    #Plota a comparação de desempenho dos modelos
    plot_model_comparisons(model_results)
    with open(os.path.join(save_dir, 'model_results.pkl'), 'wb') as f:
        pickle.dump(model_results, f)

    return model_results


<h3>Criação do modelo final utilizando os resultados do passo anterior</h3>
<p>Depois dos modelos terem sido criados e otimizados, irei criar um modelo final, que é o modelo que será usado para submeter os dados ao Kaggle. Também irei comparar visualmente o pior modelo encontrado com o melhor modelo encontrado em relação à performance com a métrica Root Mean Squared Logarithmic Error</p>

In [6]:
# Função para criar o modelo final
def cria_modelo_final(df, target_var='sales', save_dir='model_results'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    print("Preparando os dados...")
    X_train, X_test, y_train, y_test, encoders, imputer_numeric, scaler, y_scaler, train_columns = prepare_data_train(df, target_var, test_size=0.2)

    # Extrai as datas para o conjunto de teste
    dates = df.iloc[X_test.index]['date']  

    print("Treinando a baseline (Regressão Linear)...")
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    y_pred_lr = lr_model.predict(X_test)
    y_pred_lr = np.maximum(y_pred_lr, 0)
    lr_rmsle = rmsle(y_test, y_pred_lr)

    with open(os.path.join(save_dir, 'linear_regression_model.pkl'), 'wb') as f:
        pickle.dump(lr_model, f)

    print(f'Regressão Linear RMSLE: {lr_rmsle}')

    best_rmsle = lr_rmsle
    best_model = lr_model
    best_model_name = 'linear_regression_model.pkl'

    for model_file in os.listdir(save_dir):
        if model_file.endswith('_model.pkl') and model_file != 'linear_regression_model.pkl':
            model_path = os.path.join(save_dir, model_file)
            with open(model_path, 'rb') as f:
                model = pickle.load(f)

            X_test_reordered = X_test[train_columns]

            y_pred_model = model.predict(X_test_reordered)
            y_pred_model = np.maximum(y_pred_model, 0)
            model_rmsle = rmsle(y_test, y_pred_model)
            print(f'{model_file} RMSLE: {model_rmsle}')

            if model_rmsle < best_rmsle:
                best_rmsle = model_rmsle
                best_model = model
                best_model_name = model_file

    print(f'O melhor modelo é: {best_model_name} com RMSLE: {best_rmsle}')
    plot_predictions(y_test, best_model.predict(X_test[train_columns]), best_model_name, y_scaler, dates)

    return best_model, encoders, imputer_numeric, scaler, y_scaler, train_columns

<h1>Criando o arquivo de submissão</h1>
<p>Com o modelo final criado, basta apenas criar o arquivo de submissão para o Kaggle</p>

In [7]:
def prepare_and_submit(model, df_test, test, encoders, imputer_numeric, scaler, y_scaler, train_columns):
    df_test_prepared = prepare_data_test(df_test, encoders, imputer_numeric, scaler, train_columns)

    # Garantir que as colunas estejam na mesma ordem que as colunas de treinamento
    df_test_prepared = df_test_prepared[train_columns]

    # Realiza as previsões
    predictions = model.predict(df_test_prepared)
    predictions = y_scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()
    predictions = np.maximum(predictions, 0)

    # Cria e salva o arquivo de submissão
    output_filename = 'regression_submission.csv'
    submission = pd.DataFrame({'id': test['id'], 'sales': predictions})
    submission.to_csv(output_filename, index=False)

    print(f"Submission file created: {output_filename}")
    return submission


<h3>Executando todas as funções que foram criadas previamente</h3>

In [8]:
test_path = 'test.csv'
train_path = 'train.csv'
transactions_path = 'transactions.csv'
stores_path = 'stores.csv'
holidays_events_path = 'holidays_events.csv'
oil_path = 'oil.csv'
parquet_path = 'processed_data.parquet'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

filtered_train = train[train['date'] >= "2017-01-01"].copy()
filtered_train = filtered_train[filtered_train['family'].isin(test['family'])].copy()

df_transformed = feature_engineering(filtered_train, transactions_path, stores_path, holidays_events_path, oil_path, temp_dir='train_temp')
df_test = feature_engineering(test, transactions_path, stores_path, holidays_events_path, oil_path, temp_dir='test_temp')


Loading and preprocessing initial data...
Saved and cleaned up: train_temp/data.csv
Loading and saving transactions...
Saved and cleaned up: train_temp/transactions_grouped.csv
Loading and saving stores...
Saved and cleaned up: train_temp/stores.csv
Loading and saving holidays_events...
Saved and cleaned up: train_temp/holidays_events.csv
Loading, processing, and saving oil data...


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_27052\3176306971.py:83: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  oil_df_full['dcoilwtico'] = oil_df_full['dcoilwtico'].fillna(method='bfill').fillna(method='ffill')


Saved and cleaned up: train_temp/oil_df_full.csv
Loading data and merging transactions...
Merged with: train_temp/transactions_grouped.csv
Saved and cleaned up: train_temp/merged_data_transactions.csv
Loading merged data and merging stores...
Merged with: train_temp/stores.csv
Saved and cleaned up: train_temp/merged_data_stores.csv
Loading merged data and merging holidays_events...
Merged with: train_temp/holidays_events.csv
Saved and cleaned up: train_temp/merged_data_holidays_events.csv
Loading merged data and merging oil_df_full...
Merged with: train_temp/oil_df_full.csv
Adding holiday clusters and additional features...
Saved and cleaned up: train_temp/merged_df_final.csv
Loading and preprocessing initial data...
Saved and cleaned up: test_temp/data.csv
Loading and saving transactions...
Saved and cleaned up: test_temp/transactions_grouped.csv
Loading and saving stores...
Saved and cleaned up: test_temp/stores.csv
Loading and saving holidays_events...
Saved and cleaned up: test_tem

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_27052\3176306971.py:83: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  oil_df_full['dcoilwtico'] = oil_df_full['dcoilwtico'].fillna(method='bfill').fillna(method='ffill')


Saved and cleaned up: test_temp/oil_df_full.csv
Loading data and merging transactions...
Merged with: test_temp/transactions_grouped.csv
Saved and cleaned up: test_temp/merged_data_transactions.csv
Loading merged data and merging stores...
Merged with: test_temp/stores.csv
Saved and cleaned up: test_temp/merged_data_stores.csv
Loading merged data and merging holidays_events...
Merged with: test_temp/holidays_events.csv
Saved and cleaned up: test_temp/merged_data_holidays_events.csv
Loading merged data and merging oil_df_full...
Merged with: test_temp/oil_df_full.csv
Adding holiday clusters and additional features...
Saved and cleaned up: test_temp/merged_df_final.csv


In [9]:
model_results = run_optimized_models(df_transformed)

Preparando os dados...


DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class

In [ ]:
# Preparar os dados de treinamento e criar o modelo final
best_model, encoders, imputer_numeric, scaler, y_scaler, train_columns = cria_modelo_final(df_transformed)


In [ ]:
# Preparar os dados de teste e criar a submissão
submission = prepare_and_submit(best_model, df_test, test, encoders, imputer_numeric, scaler, y_scaler, train_columns)
